In [1]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

In [2]:
bids_root = pathlib.Path('D:/研究生/mne/mne_data/out_data/sample_BIDS/')
bids_path = mne_bids.BIDSPath(subject="01", 
                             session="01", 
                             task="audiovisual",
                             run="01",
                             datatype='meg',
                             root=bids_root)
raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)
events, event_id = mne.events_from_annotations(raw)

Opening raw data file D:\研究生\mne\mne_data\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading events from D:\研究生\mne\mne_data\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.tsv.
Reading channel info from D:\研究生\mne\mne_data\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Reading 0 ... 166799  =      0.000 ...   277.714 secs...


<ipython-input-2-51eabae648ce>:8: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 19821 samples (33.001 sec)

Used Annotations descriptions: ['Auditory/Left', 'Auditory/Right', 'Button', 'Smiley', 'Visual/Right', 'Visual/left']


In [3]:
event_id

{'Auditory/Left': 1,
 'Auditory/Right': 2,
 'Button': 3,
 'Smiley': 4,
 'Visual/Right': 5,
 'Visual/left': 6}

### 指定epoch、event的起点、终点，单位是s
### 对于baseline的解释：当使用baseline矫正的时候，将一个时间间隔看作是“baseline”。
### 如果值为None，不会使用baseline矫正，我的理解是在tmin和tmax上面的一个修正

In [4]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0)

epochs = mne.Epochs(raw, 
                   events=events,
                   event_id=event_id,
                   tmin=tmin,
                   tmax=tmax, 
                   baseline=baseline,
                   preload=True)

epochs

Not setting metadata
Not setting metadata
320 matching events found
Setting baseline interval to [-0.2996928197375818, 0.0] sec
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Loading data for 320 events and 481 original time points ...
0 bad epochs dropped


Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Right: 71Visual/left: 73
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [5]:
epochs.plot(events=events, event_id=event_id)

<MNEBrowseFigure size 1920x986 with 4 Axes>

In [6]:
print(epochs)

<Epochs |  320 events (all good), -0.299693 - 0.499488 sec, baseline -0.299693 – 0 sec, ~444.8 MB, data loaded,
 'Auditory/Left': 72
 'Auditory/Right': 73
 'Button': 16
 'Smiley': 15
 'Visual/Right': 71
 'Visual/left': 73>


### 基于实验情况挑选epoch

In [7]:
epochs["Auditory/Left"]

Number of events,72
Events,Auditory/Left: 72
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [8]:
epochs["Auditory"]

Number of events,145
Events,Auditory/Left: 72Auditory/Right: 73
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [9]:
epochs["Visual"].plot_image()
# 这个图可以得到epoch及它的GFP

Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"
combining channels using "gfp"
combining channels using "gfp"
Dropped 0 epochs: 
Channels marked as bad: ['MEG 2443', 'EEG 053']


[<Figure size 1280x960 with 3 Axes>,
 <Figure size 1280x960 with 3 Axes>,
 <Figure size 1280x960 with 3 Axes>]

In [10]:
epochs.copy().pick_types(meg=False, eeg=True).plot_image()

Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
Not setting metadata
Not setting metadata
320 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"


[<Figure size 1280x960 with 3 Axes>]

<div class="alert alert-success">
   <b>保存数据</b>
   <ul>
        <li>condition为right时，包含了所有右边的事件，听觉的、视觉的</li>
   </ul>
</div>

In [11]:
epochs.save(pathlib.Path('D:\研究生\mne\mne_data\out_data') / 'epoch_epo.fif', overwrite=True)


Overwriting existing file.


In [21]:
evoked_auditory = epochs['Auditory'].average()
evoked_visual = epochs['Visual'].average()

In [22]:
evoked_auditory.plot(spatial_colors=True)

<Figure size 1920x918 with 6 Axes>

In [23]:
evoked_auditory.plot_topomap(ch_type='mag', times=[0.05, 0.10, 0.15, 0.20])

<MNEFigure size 1500x440 with 5 Axes>

In [25]:
#joint image 上面两种图的结合
evoked_auditory.plot_joint(picks='mag')

Projections have already been applied. Setting proj attribute to True.


<Figure size 1600x840 with 7 Axes>

In [27]:
mne.viz.plot_compare_evokeds([evoked_auditory, evoked_visual], picks='mag')

combining channels using "gfp"
combining channels using "gfp"


[<Figure size 1920x918 with 1 Axes>]

<div class="alert alert-success">
   <b>比较图</b>
   <ul>
       <li>通道选择EEG信号，比较选择左边和右边视觉的刺激</li>
   </ul>
</div>

In [34]:
evoked_visual_left = epochs['Visual/left'].average()
evoked_visual_right = epochs['Visual/Right'].average()

In [38]:
mne.viz.plot_compare_evokeds([evoked_visual_left, evoked_visual_right], picks='mag')

combining channels using "gfp"
combining channels using "gfp"


[<Figure size 1920x918 with 1 Axes>]

In [37]:
evoked_visual_left

<Evoked | 'Visual/left' (average, N=73), -0.29969 – 0.49949 sec, baseline -0.299693 – 0 sec, 366 ch, ~4.6 MB>

### 保存evoke数据

In [40]:
mne.write_evokeds(fname=pathlib.Path('D:\研究生\mne\mne_data\out_data') / 'evokeds_ave.fif', 
                 evoked=[evoked_auditory, evoked_visual])

### 读取evoke数据

In [41]:
evokeds = mne.read_evokeds(fname=pathlib.Path('D:\研究生\mne\mne_data\out_data') / 'evokeds_ave.fif')
evokeds

Reading D:\研究生\mne\mne_data\out_data\evokeds_ave.fif ...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
    Found the data of interest:
        t =    -299.69 ...     499.49 ms (0.50 × Auditory/Left + 0.50 × Auditory/Right)
        0 CTF compensation matrices available
        nave = 145 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Loaded Evoked data is baseline-corrected (baseline: [-0.299693, 0] sec)
    Read a total of 3 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
    Found the data of interest:
        t =    -299.69 ...     499.49 ms (0.49 × Visual/Right + 0.51 × Visual/left)
        0 CTF compensation matrices available
        nave = 144 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Loaded Evoked data is baseline-corrected (baseline:

[<Evoked | '0.50 × Auditory/Left + 0.50 × Auditory/Right' (average, N=145), -0.29969 – 0.49949 sec, baseline -0.299693 – 0 sec, 366 ch, ~4.6 MB>,
 <Evoked | '0.49 × Visual/Right + 0.51 × Visual/left' (average, N=144), -0.29969 – 0.49949 sec, baseline -0.299693 – 0 sec, 366 ch, ~4.6 MB>]

In [42]:
evokeds[0]

<Evoked | '0.50 × Auditory/Left + 0.50 × Auditory/Right' (average, N=145), -0.29969 – 0.49949 sec, baseline -0.299693 – 0 sec, 366 ch, ~4.6 MB>

In [48]:
evokeds = mne.read_evokeds(fname=pathlib.Path('D:\研究生\mne\mne_data\out_data') / 'evokeds_ave.fif',
                          condition='0.50 × Auditory/Left + 0.50 × Auditory/Right')


Reading D:\研究生\mne\mne_data\out_data\evokeds_ave.fif ...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
    Found the data of interest:
        t =    -299.69 ...     499.49 ms (0.50 × Auditory/Left + 0.50 × Auditory/Right)
        0 CTF compensation matrices available
        nave = 145 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Loaded Evoked data is baseline-corrected (baseline: [-0.299693, 0] sec)
